### Day 1: Ingest and Index Your Data

In [7]:
import io
import zipfile
import requests
import frontmatter

def read_repo_data(repo_owner, repo_name, branch='main'):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
        branch: Branch name (default: 'main', some repos use 'master')
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/{branch}'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md') 
            or filename_lower.endswith('.mdx')):
            continue
    
        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue
    
    zf.close()
    return repository_data

In [8]:
# VS Code documentation
mcs = read_repo_data('microsoft', 'vscode-docs')
print(f"Downloaded: {len(mcs)} documents")

# Hugging Face Transformers
hgf = read_repo_data('huggingface', 'transformers')
print(f"Downloaded: {len(hgf)} documents")

# Python documentation
pyth = read_repo_data('python', 'cpython')
print(f"Downloaded: {len(pyth)} documents")

# LangChain uses 'master' branch
my_docs = read_repo_data('langchain-ai', 'langchain', branch='master')
print(f"Downloaded: {len(my_docs)} documents")

Downloaded: 744 documents
Downloaded: 1227 documents
Downloaded: 40 documents
Downloaded: 36 documents
